In [1]:
import numpy as np
import pandas as pd
from implicit.datasets.lastfm import get_lastfm
from implicit.nearest_neighbours import bm25_weight
from implicit.cpu.bpr import BayesianPersonalizedRanking

/Users/tgayanvitharanagamage/anaconda3/envs/rec-sys-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Experiment with the given example dataset

In [2]:
artists, users, artist_user_plays = get_lastfm()

In [3]:
type(artist_user_plays)

scipy.sparse._csr.csr_matrix

In [4]:
# weight the matrix, both to reduce impact of users that have played the same artist thousands of times
# and to reduce the weight given to popular items
# https://benfred.github.io/implicit/tutorial_lastfm.html#:~:text=The%20first%20step,classic%20information%20retrieval%3A
# https://en.wikipedia.org/wiki/Okapi_BM25
artist_user_plays = bm25_weight(artist_user_plays, K1=100, B=0.8)

# get the transpose since the most of the functions in implicit expect (user, item) sparse matrices instead of (item, user)
user_plays = artist_user_plays.T.tocsr()

In [6]:
model = BayesianPersonalizedRanking(factors=64, regularization=0.01, learning_rate=0.1, iterations=15, num_threads=0)
model.fit(user_plays)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [09:12<00:00, 36.86s/it, train_auc=95.25%, skipped=2.18%]


In [7]:
# Get recommendations for the a single user
userid = 12345
ids, scores = model.recommend(userid, user_plays[userid], N=10, filter_already_liked_items=False)
pd.DataFrame({"artist": artists[ids], "score": scores, "already_liked": np.in1d(ids, user_plays[userid].indices)})

,artist,score,already_liked
0,laibach,4.972835,False
1,mortiis,4.710769,True
2,sopor aeternus & the ensemble of shadows,4.707899,False
3,the sisters of mercy,4.629304,False
4,fields of the nephilim,4.560995,False
5,the red army choir,4.531089,False
6,za frûmi,4.491640,False
7,coptic rain,4.488866,False
8,christian death,4.481449,False
9,welle:erdball,4.477931,False


In [8]:
# get related items for the beatles (itemid = 25512)
ids, scores= model.similar_items(252512)

# display the results using pandas for nicer formatting
pd.DataFrame({"artist": artists[ids], "score": scores})

,artist,score
0,the beatles,1.000000
1,paul mccartney,0.793470
2,the beach boys,0.756167
3,ahoud banai,0.755275
4,the british blues quintet,0.748916
5,orville hammond trio,0.742344
6,movie cast,0.738473
7,lovin spoonful,0.735691
8,aljazeeraenglish,0.735300
9,byron t,0.734515


In [ ]:
# Make recommendations for the first 1000 users in the dataset
userids = np.arange(1000)
ids, scores = model.recommend(userids, user_plays[userids])
ids, ids.shape

## Experiment with amazon beauty dataset

In [2]:
import numpy as np
import pandas as pd
from implicit.datasets.lastfm import get_lastfm
from implicit.nearest_neighbours import bm25_weight
from implicit.cpu.bpr import BayesianPersonalizedRanking
from utils import pandas_df_to_csr

In [2]:
amazon_beauty_df = pd.read_csv("ratings_Beauty.csv")

In [3]:
user_map, item_map, amazon_beauty_csr = pandas_df_to_csr(amazon_beauty_df)

In [4]:
amazon_beauty_csr_bm25 = bm25_weight(amazon_beauty_csr, K1=100, B=0.8)

In [5]:
amazon_beauty_csr_bm25 = amazon_beauty_csr_bm25.tocsr()

In [6]:
model = BayesianPersonalizedRanking(factors=64, regularization=0.01, learning_rate=0.1, iterations=15, num_threads=0)
model.fit(amazon_beauty_csr_bm25)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:45<00:00,  3.04s/it, train_auc=91.64%, skipped=0.03%]


In [7]:
# Get recommendations for the a single user
userid = 725046
ids, scores = model.recommend(userid, amazon_beauty_csr_bm25[userid], N=10, filter_already_liked_items=False)

In [8]:
pd.DataFrame({"ProductId": item_map.loc[ids]["ProductId"], "score": scores, "already_purchased": np.in1d(ids, amazon_beauty_csr_bm25[userid].indices)})

,ProductId,score,already_purchased
ItemIndex,,,
81854,B002OVV7F0,2.696701,True
62199,B001H928KI,2.067228,False
64175,B001KYPZRS,2.022594,False
98082,B003HFSZ3Y,1.952995,False
68981,B001R66FFA,1.947325,False
68342,B001Q88FO8,1.925719,False
113434,B0047PPO0U,1.914744,False
203415,B00B1ZRTP2,1.899756,False
98092,B003HG4VLS,1.854516,False


In [13]:
item_map.loc[ids]["ProductId"]

ItemIndex
114276    B0049WJA9C
65766     B001MA0QY2
181492    B008O4YM4Y
10253     B0009PVV40
103349    B003S516XO
174368    B0085WHBHU
75617     B002B9DWBC
89252     B00325D0WK
73976     B0027A7CLG
181069    B008MP481M
Name: ProductId, dtype: object

## End of Experiments

In [1]:
import numpy as np
import pandas as pd
import gc
from implicit.datasets.lastfm import get_lastfm
from implicit.nearest_neighbours import bm25_weight
from implicit.cpu.bpr import BayesianPersonalizedRanking
from implicit import evaluation
from utils import pandas_df_to_csr

/Users/tgayanvitharanagamage/anaconda3/envs/rec-sys-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Pre-processing

In [2]:
amazon_beauty_df = pd.read_csv("ratings_Beauty.csv")

In [3]:
# Convert pandas df to CSR format
# user_map and item_map contains index in csr format to original id mappings for users and items repectively
user_map, item_map, amazon_beauty_csr = pandas_df_to_csr(amazon_beauty_df)

In [4]:
# weight the matrix, both to reduce impact of users that have puchased the same item thousands of times
# and to reduce the weight given to popular items
# Output is a COO matrix
amazon_beauty_coo_bm25 = bm25_weight(amazon_beauty_csr, K1=100, B=0.8)

### Train-test split

In [5]:
train_coo, test_coo = evaluation.train_test_split(amazon_beauty_coo_bm25, train_percentage=0.8, random_state=55)
print(f"Train size: {train_coo.size} \n Test size: {test_coo.size}")

Train size: 1618938 
 Test size: 404132


In [6]:
#Convert coo to csr
train_csr = train_coo.tocsr()
test_csr = test_coo.tocsr()

## Training

In [7]:
model = BayesianPersonalizedRanking(factors=64, regularization=0.01, learning_rate=0.1, iterations=15, num_threads=0)
model.fit(train_csr)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [01:32<00:00,  6.17s/it, train_auc=89.31%, skipped=0.03%]


## Evaluation

In [8]:
ranking_metrics_at_10 = evaluation.ranking_metrics_at_k(model, train_csr, test_csr, K=10, show_progress=True, num_threads=0)
ranking_metrics_at_10

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 330846/330846 [00:48<00:00, 6782.90it/s]


{'precision': 0.005786480609187128,
 'map': 0.002408520120430416,
 'ndcg': 0.003102807988801171,
 'auc': 0.5022218212995132}

### Save model

In [9]:
model.save("5_CF_BPR_implicit")

## Hyper-parameter Tuning

In [3]:
# Hyper-parameters lists
latent_factors = [32, 64, 128, 256]
regularization = [0.005, 0.01, 0.05, 0.1]
learning_rate = [0.05, 0.1, 0.5]

In [18]:
results = []
for f in latent_factors:
    for r in regularization:
        for l in learning_rate:
            print("Training model with below parameter values --------")
            print(f"latent_factors: {f}, regularization: {r}, learning_rate: {l}")
            model =  BayesianPersonalizedRanking(factors=f, regularization=r, learning_rate=l, iterations=15, num_threads=0)
            model.fit(train_csr)
            ranking_metrics_at_10 = evaluation.ranking_metrics_at_k(model, train_csr, test_csr, K=10, show_progress=True, num_threads=0)
            print("Evaluation results: \n", ranking_metrics_at_10)
            results.append(
                (f,r,l,ranking_metrics_at_10['precision'],ranking_metrics_at_10['map'],ranking_metrics_at_10['ndcg'],ranking_metrics_at_10['auc'])
            )
            # Garbage handling
            del model
            gc.collect()

Training model with below parameter values --------
latent_factors: 32, regularization: 0.005, learning_rate: 0.05


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 330846/330846 [00:54<00:00, 6056.03it/s]


Evaluation results: 
 {'precision': 0.00510146891416928, 'map': 0.0015033694633245942, 'ndcg': 0.0021315162558968215, 'auc': 0.5016153388481799}
Training model with below parameter values --------
latent_factors: 32, regularization: 0.005, learning_rate: 0.1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 330846/330846 [00:54<00:00, 6077.19it/s]


Evaluation results: 
 {'precision': 0.005467638802051548, 'map': 0.0022934639351212148, 'ndcg': 0.0029528598907335073, 'auc': 0.5021309568635243}
Training model with below parameter values --------
latent_factors: 32, regularization: 0.005, learning_rate: 0.5


...

Evaluation results: 
 {'precision': 0.0003611879846457742, 'map': 3.2887819624322404e-05, 'ndcg': 6.666347243447022e-05, 'auc': 0.5000213874473228}
Training model with below parameter values --------
latent_factors: 256, regularization: 0.1, learning_rate: 0.1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 330846/330846 [02:10<00:00, 2544.39it/s]


Evaluation results: 
 {'precision': 0.0010860549055555695, 'map': 0.00012439657776212735, 'ndcg': 0.00024543475035451707, 'auc': 0.5001835229285885}
Training model with below parameter values --------
latent_factors: 256, regularization: 0.1, learning_rate: 0.5


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 330846/330846 [02:16<00:00, 2418.09it/s]

Evaluation results: 
 {'precision': 0.002969214328950089, 'map': 0.0011324188937991337, 'ndcg': 0.0015375179012410384, 'auc': 0.5011640557198422}


In [16]:
results_df = pd.DataFrame(results, columns=['latent_factors', 'regularization', 'learning_rate', 'precision', 'map', 'ndcg', 'auc'])
results_df

,latent_factors,regularization,learning_rate,precision,map,ndcg,auc
0,32,0.005,0.05,0.005101,0.001503,0.002132,0.501615
1,32,0.005,0.10,0.005468,0.002293,0.002953,0.502131
2,32,0.005,0.50,0.001188,0.000249,0.000432,0.500398
3,32,0.010,0.05,0.004481,0.001051,0.001614,0.501302
4,32,0.010,0.10,0.005216,0.002069,0.002711,0.501991
5,32,0.010,0.50,0.002959,0.001068,0.001470,0.501147
6,32,0.050,0.05,0.001594,0.000246,0.000406,0.500254
7,32,0.050,0.10,0.002172,0.000454,0.000701,0.500471
8,32,0.050,0.50,0.003741,0.001479,0.001989,0.501543
9,32,0.100,0.05,0.000473,0.000045,0.000091,0.500035


In [17]:
results_df.to_csv('hyper_parameter_training_results_BPR.csv')

## Temp code to retrieve results from text to list

In [5]:
params = []
for f in latent_factors:
    for r in regularization:
        for l in learning_rate:
            params.append([f,r,l])

In [7]:
with open("tmp_BPR_results.txt") as fp:
    text = fp.read()
text

"[\n    {'precision': 0.00510146891416928, 'map': 0.0015033694633245942, 'ndcg': 0.0021315162558968215, 'auc': 0.5016153388481799},\n    {'precision': 0.005467638802051548, 'map': 0.0022934639351212148, 'ndcg': 0.0029528598907335073, 'auc': 0.5021309568635243},\n    {'precision': 0.001188183921903685, 'map': 0.00024903875138810374, 'ndcg': 0.00043161308825532207, 'auc': 0.5003978845943127},\n    {'precision': 0.004481221961225847, 'map': 0.001050715130654907, 'ndcg': 0.0016142223690655615, 'auc': 0.5013023857993317},\n    {'precision': 0.005216052688608629, 'map': 0.0020690025506611494, 'ndcg': 0.002711173039869431, 'auc': 0.50199076293975},\n    {'precision': 0.002959250522477102, 'map': 0.0010683458401969641, 'ndcg': 0.0014697439319069454, 'auc': 0.5011472774375025},\n    {'precision': 0.0015942090356779, 'map': 0.00024641180358203304, 'ndcg': 0.00040603108998945816, 'auc': 0.500253743824585},\n    {'precision': 0.002172109811111139, 'map': 0.0004536812015142101, 'ndcg': 0.0007014983

In [14]:
 metrics = eval(text)

In [15]:
results = []
for p,r in zip(params, metrics):
    results.append(
        (p[0], p[1], p[2], r['precision'], r['map'], r['ndcg'], r['auc'])
    )
results 

[(32,
  0.005,
  0.05,
  0.00510146891416928,
  0.0015033694633245942,
  0.0021315162558968215,
  0.5016153388481799),
 (32,
  0.005,
  0.1,
  0.005467638802051548,
  0.0022934639351212148,
  0.0029528598907335073,
  0.5021309568635243),
 (32,
  0.005,
  0.5,
  0.001188183921903685,
  0.00024903875138810374,
  0.00043161308825532207,
  0.5003978845943127),
 (32,
  0.01,
  0.05,
  0.004481221961225847,
  0.001050715130654907,
  0.0016142223690655615,
  0.5013023857993317),
 (32,
  0.01,
  0.1,
  0.005216052688608629,
  0.0020690025506611494,
  0.002711173039869431,
  0.50199076293975),
 (32,
  0.01,
  0.5,
  0.002959250522477102,
  0.0010683458401969641,
  0.0014697439319069454,
  0.5011472774375025),
 (32,
  0.05,
  0.05,
  0.0015942090356779,
  0.00024641180358203304,
  0.00040603108998945816,
  0.500253743824585),
 (32,
  0.05,
  0.1,
  0.002172109811111139,
  0.0004536812015142101,
  0.0007014983008247475,
  0.5004708951330932),
 (32,
  0.05,
  0.5,
  0.0037414093306065714,
  0.0014